In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
import pandas as pd
import numpy as np
# do the LDA
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
import pprint
import nltk
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
with open('Feature_List', newline='') as f:
    reader = csv.reader(f)
    feature_list = list(reader)

In [3]:
BA_feature=feature_list[0]
DA_feature=feature_list[1]
DS_feature=feature_list[2]
DE_feature=feature_list[3]

In [19]:
all_feature=BA_feature+DA_feature+DS_feature+DE_feature

In [20]:
len(all_feature)

80

In [21]:
len(set(all_feature))

39

In [22]:
BA_JD=pd.read_csv("BA_JD.csv")[0:5]

In [23]:
clean=["we","our","us","itself","employees","firm","apply","disabilities",
       "re","company","description","accomendation","\nWe","relocation","location"]
stop_words = set(stopwords.words('english')) 
def clean_JD(jd,name):
    name=[x.lower() for x in word_tokenize(name) if x.isalnum()]
    if len(name)>1:
        s=""
        try:
            for i in name:
                s+=i[0]
            name+=[s]
        except:
            pass
    res=""
    for sentence in sent_tokenize(jd):
        for j in sentence.split("\n"):
            if not j: continue
            sent=[]
            for x in word_tokenize(j):
                if x.isalnum():
                    sent.append(x.lower())
            A=set(sent)
            B=set(name+clean)
            if not A&B:
                res+=j+"\n"
    word_tokens = word_tokenize(res)
    wordlist=[word for word in word_tokens if word.isalnum() and word not in stop_words]
    full_text=""
    for word in wordlist:
        full_text+=word+" "
    
    return full_text

In [24]:
BA_JD['details'] =BA_JD.apply(lambda x: clean_JD(x['details'],x['company']),axis=1)

In [25]:
BA_JD

,Unnamed: 0,position,company,location,details
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted Azhar Ali Khan Send InMail Business Ana...
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted Khalid Ansari Send InMail Eligibility C...
2,2,Business Analyst,Cisco,"New York, NY",Who You Are As Security Licensing Specialist a...
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Job Id 22461117 The analyst work closely Finan...
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Responsibilities vary depending assigned proje...


In [26]:
BA_JD['details'].iloc[1]

'Posted Khalid Ansari Send InMail Eligibility Criteria Your undergraduate graduation year 2022 later You working towards degree Job Requirements Minimum GPA strongly preferred may consider Work Authorization Applicants must legally authorized employment United States without need current future work authorization Desire work within United States '

In [27]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()

# process the keywords
ba_feature_set = set([ps.stem(tok) for tok in BA_feature]) 
da_feature_set = set([ps.stem(tok) for tok in DA_feature]) 
ds_feature_set = set([ps.stem(tok) for tok in DS_feature]) 
de_feature_set = set([ps.stem(tok) for tok in DE_feature])
all_feature_set = set([ps.stem(tok) for tok in all_feature])

In [28]:
# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    desc=desc.replace("-"," ")
    desc=desc.replace("/"," ")
    tokens = [x for x in word_tokenize(desc) if x.isalnum()]
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(tok).lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

In [29]:
BA_JD

,Unnamed: 0,position,company,location,details
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted Azhar Ali Khan Send InMail Business Ana...
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted Khalid Ansari Send InMail Eligibility C...
2,2,Business Analyst,Cisco,"New York, NY",Who You Are As Security Licensing Specialist a...
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Job Id 22461117 The analyst work closely Finan...
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Responsibilities vary depending assigned proje...


In [30]:
def mapping_list(df):
    #name=get_df_name(df)
    #df['details']=name.split("_")[1]
    df['job_description_word_set'] =df['details'].map(prepare_job_desc)
    
    ba_feature = []
    da_feature = []
    ds_feature = []
    de_feature = []
    all_feature = []
   

    num_postings = len(df.index)
    for i in range(num_postings):
        job_desc = df.iloc[i]['details'].lower()
        job_desc_set = df.iloc[i]['job_description_word_set']

        # check if the keywords are in the job description. Look for exact match by token.
        ba = ba_feature_set.intersection(job_desc_set)
        da = da_feature_set.intersection(job_desc_set)
        ds = ds_feature_set.intersection(job_desc_set)
        de = de_feature_set.intersection(job_desc_set)
        al = all_feature_set.intersection(job_desc_set)

        
        if len(ba) == 0:
            ba=[None]
        if len(da) == 0:
            da=[None]
        if len(ds) == 0:
            ds=[None]
        if len(de) == 0:
            de=[None]
        if len(al) == 0:
            al=[None]
    
                
        ba_feature.append(list(ba))
        da_feature.append(list(da))
        ds_feature.append(list(ds))
        de_feature.append(list(de))
        all_feature.append(list(al))
    
    return ba_feature, da_feature, ds_feature, de_feature,all_feature

In [31]:
def df_with_feature(df):
    ba, da, ds, de, al= mapping_list(df)
    df["ba"]=ba
    df["da"]=da
    df["ds"]=ds
    df["de"]=ds
    df["all"]=al
   
    return df

In [32]:
ba_final=df_with_feature(BA_JD)
ba_final

,Unnamed: 0,position,company,location,details,job_description_word_set,ba,da,ds,de,all
0,0,Business Analyst Intern,RM Talent Solutions,"Washington, United States",Posted Azhar Ali Khan Send InMail Business Ana...,"{requir, way, busi, run, azhar, techniqu, gpa,...","[commun, busi, data]","[commun, busi, data]","[techniqu, busi, data]","[techniqu, busi, data]","[commun, techniqu, busi, data]"
1,1,Business Analyst Intern,RM Talent Solutions,"Maine, United States",Posted Khalid Ansari Send InMail Eligibility C...,"{requir, desir, ansari, 2022, gpa, job, prefer...",[None],[None],[None],[None],[None]
2,2,Business Analyst,Cisco,"New York, NY",Who You Are As Security Licensing Specialist a...,"{product, detail, busi, follow, need, process,...","[engin, commun, busi, excel]","[busi, commun, engin, excel, cloud]","[busi, excel]","[busi, excel]","[busi, commun, engin, excel, cloud]"
3,3,U.S. Private Banking & Wealth Business Analyti...,Citi,"New York, NY",Job Id 22461117 The analyst work closely Finan...,"{law, product, busi, guidelin, sql, analyst, p...","[busi, program, commun, statist, sql, excel, p...","[busi, program, commun, statist, sql, excel, p...","[busi, statist, techniqu, excel, python, data,...","[busi, statist, techniqu, excel, python, data,...","[busi, program, commun, techniqu, sql, statist..."
4,4,Business Analyst Intern,2 a.m. Talent,"Jacksonville, FL",Responsibilities vary depending assigned proje...,"{respons, present, requir, busi, qualif, tool,...","[commun, busi, excel]","[commun, busi, excel]","[busi, excel]","[busi, excel]","[commun, busi, excel]"
